<a href="https://colab.research.google.com/github/Kekoa-Pickett/ML-HW2/blob/main/HW2_NB_%26_SVMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories.

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work.

In [48]:
import platform
display(platform.system())
import os
file_download_link = 'https://www.dropbox.com/scl/fi/j1dxtqjerbdbl81e05nvp/hw4data.zip?rlkey=8mkxz4j8lngziok6on782a8u4&dl=0'
if os.name == 'nt':
    print('Please download your dataset here:', file_download_link)
else:
    # We need to first download the data here:
    !wget -O data.zip "$file_download_link" -o /dev/null
    !unzip data.zip > /dev/null

'Linux'

replace garments_worker_productivity.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [49]:
!sed 's/,/\t/g' garments_worker_productivity.csv > garments_worker_productivity.tsv

In [50]:
# If your data is on google drive then uncomment the code below to access
# your google drive.
#from google.colab import drive
#drive.mount('/content/drive')

In [51]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`.

Realize that `quarter` is not referring to certain months of the year, but certain days of the month. "Quarter1" represents days 1-7, "Quarter2" days 8-14, "Quarter3" days 15-21, "Quarter4" days 22-28, and "Quarter5" days 29-31

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - You must use `df.describe()` or `df.info()` to display the data after preprocessing. **No credit** will be given if this step is omitted.
 - Fill the empty values with 0.


In [60]:
import pandas as pd
# If put the data(.csv) under the same folder, you could use
# df = pd.read_csv('./garments_worker_productivity.csv')
df = pd.read_csv('garments_worker_productivity.tsv', sep='\t')
df.drop(columns=['date'], inplace=True)
for i in ['day', 'quarter', 'department', 'team']:
    print(df[i].unique())
df.replace('sweing', 'sewing', inplace=True)
df.replace('finishing ', 'finishing', inplace=True)
df['satisfied'] = df.apply(lambda x: 1 if x['actual_productivity'] >= x['targeted_productivity'] else 0, axis=1)
df.drop(columns=['actual_productivity', 'targeted_productivity'], inplace=True)
for i in df.columns:
    if df[i].isna().any():
        print(i)
df.fillna(0, inplace=True)
df.describe()

['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
['sweing' 'finishing ' 'finishing']
[ 8  1 11 12  6  7  2  3  9 10  5  4]
wip


,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
count,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000
mean,6.426901,15.062172,687.228070,4567.460317,38.210526,0.730159,0.369256,0.150376,34.609858,0.730994
std,3.463963,10.943219,1514.582341,3348.823563,160.182643,12.709757,3.268987,0.427848,22.197687,0.443628
min,1.000000,2.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
25%,3.000000,3.940000,0.000000,1440.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000
50%,6.000000,15.260000,586.000000,3960.000000,0.000000,0.000000,0.000000,0.000000,34.000000,1.000000
75%,9.000000,24.260000,1083.000000,6960.000000,50.000000,0.000000,0.000000,0.000000,57.000000,1.000000
max,12.000000,54.560000,23122.000000,25920.000000,3600.000000,300.000000,45.000000,2.000000,89.000000,1.000000


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attributes**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.
 - You **must** show the first five row of your encoded dataset, as well as the shape of your train test split. **No credit** will be given if this step is omitted.

In [61]:
# Remember to continue the task with your processed data from Exercise 1
df['day'] = pd.Categorical(df['day']).codes
df['quarter'] = pd.Categorical(df['quarter']).codes
df['department'] = pd.Categorical(df['department']).codes
df['team'] = pd.Categorical(df['team']).codes
train = df.sample(frac=0.8, random_state=0)
test = df.drop(train.index)
print(df.head())
print("train shape: ", train.shape)
print("test shape: ", test.shape)

   quarter  department  day  team    smv     wip  over_time  incentive  \
0        0           1    3     7  26.16  1108.0       7080         98   
1        0           0    3     0   3.94     0.0        960          0   
2        0           1    3    10  11.41   968.0       3660         50   
3        0           1    3    11  11.41   968.0       3660         50   
4        0           1    3     5  25.90  1170.0       1920         50   

   idle_time  idle_men  no_of_style_change  no_of_workers  satisfied  
0        0.0         0                   0           59.0          1  
1        0.0         0                   0            8.0          1  
2        0.0         0                   0           30.5          1  
3        0.0         0                   0           30.5          1  
4        0.0         0                   0           56.0          1  
train shape:  (958, 13)
test shape:  (239, 13)


### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [62]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report
model = CategoricalNB()
model.fit(train[['day', 'quarter', 'department', 'team']], train['satisfied'])
print(classification_report(test['satisfied'], model.predict(test[['day', 'quarter', 'department', 'team']])))

              precision    recall  f1-score   support

           0       0.57      0.18      0.28        71
           1       0.73      0.94      0.82       168

    accuracy                           0.72       239
   macro avg       0.65      0.56      0.55       239
weighted avg       0.68      0.72      0.66       239



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [63]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model = GaussianNB()
model.fit(scaler.fit_transform(train.drop(columns=['day', 'quarter', 'team','satisfied'])), train['satisfied'])
print(classification_report(test['satisfied'], model.predict(scaler.transform(test.drop(columns=['day', 'quarter', 'team','satisfied'])))))


              precision    recall  f1-score   support

           0       0.71      0.14      0.24        71
           1       0.73      0.98      0.83       168

    accuracy                           0.73       239
   macro avg       0.72      0.56      0.53       239
weighted avg       0.72      0.73      0.66       239



## Exercies 3 - SVM Classifier (35 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attributes**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.
 - You **must** show the first five row of your encoded dataset, as well as the shape of your train test split. **No credit** will be given if this step is omitted.

In [64]:
# Remember to continue the task with your processed data from Exercise 1
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(df[['day', 'quarter', 'department', 'team']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['day', 'quarter', 'department', 'team'])) #create a DataFrame from the encoded data
df = df.drop(columns=['day', 'quarter', 'department', 'team'])
df = pd.concat([encoded_df, df], axis=1)

train = df.sample(frac=0.8, random_state=0)
test = df.drop(train.index)
print(df.head())
print("train shape: ", train.shape)
print("test shape: ", test.shape)

   day_0  day_1  day_2  day_3  day_4  day_5  quarter_0  quarter_1  quarter_2  \
0    0.0    0.0    0.0    1.0    0.0    0.0        1.0        0.0        0.0   
1    0.0    0.0    0.0    1.0    0.0    0.0        1.0        0.0        0.0   
2    0.0    0.0    0.0    1.0    0.0    0.0        1.0        0.0        0.0   
3    0.0    0.0    0.0    1.0    0.0    0.0        1.0        0.0        0.0   
4    0.0    0.0    0.0    1.0    0.0    0.0        1.0        0.0        0.0   

   quarter_3  ...  team_11    smv     wip  over_time  incentive  idle_time  \
0        0.0  ...      0.0  26.16  1108.0       7080         98        0.0   
1        0.0  ...      0.0   3.94     0.0        960          0        0.0   
2        0.0  ...      0.0  11.41   968.0       3660         50        0.0   
3        0.0  ...      1.0  11.41   968.0       3660         50        0.0   
4        0.0  ...      0.0  25.90  1170.0       1920         50        0.0   

   idle_men  no_of_style_change  no_of_workers  sa

### Exercise 3.2 - SVM with Different Kernels (15 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [66]:
# Remember to do this task with your processed data from Exercise 3.1
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model = SVC(kernel='linear')
model.fit(scaler.fit_transform(train.drop(columns=['satisfied'])), train['satisfied'])
print("linear: ", classification_report(test['satisfied'], model.predict(scaler.transform(test.drop(columns=['satisfied'])))))
model = SVC(kernel='rbf')
model.fit(scaler.fit_transform(train.drop(columns=['satisfied'])), train['satisfied'])
print("rbf: ", classification_report(test['satisfied'], model.predict(scaler.transform(test.drop(columns=['satisfied'])))))

linear:                precision    recall  f1-score   support

           0       0.88      0.10      0.18        71
           1       0.72      0.99      0.84       168

    accuracy                           0.73       239
   macro avg       0.80      0.55      0.51       239
weighted avg       0.77      0.73      0.64       239

rbf:                precision    recall  f1-score   support

           0       0.62      0.28      0.39        71
           1       0.75      0.93      0.83       168

    accuracy                           0.74       239
   macro avg       0.69      0.61      0.61       239
weighted avg       0.72      0.74      0.70       239



### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class.
 - Oversample the **training data**.
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br >
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [70]:
# Remember to do this task with your processed data from Exercise 3.
from imblearn.over_sampling import RandomOverSampler
print(train['satisfied'].value_counts())
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(train.drop(columns=['satisfied']), train['satisfied'])
print(pd.Series(y_resampled).value_counts())
scaler = StandardScaler()
model = SVC(kernel='linear')
model.fit(scaler.fit_transform(X_resampled), y_resampled)
print("linear: ", classification_report(test['satisfied'], model.predict(scaler.transform(test.drop(columns=['satisfied'])))))
model = SVC(kernel='rbf')
model.fit(scaler.fit_transform(X_resampled), y_resampled)
print("rbf: ", classification_report(test['satisfied'], model.predict(scaler.transform(test.drop(columns=['satisfied'])))))

satisfied
1    707
0    251
Name: count, dtype: int64
satisfied
1    707
0    707
Name: count, dtype: int64
linear:                precision    recall  f1-score   support

           0       0.47      0.68      0.55        71
           1       0.83      0.67      0.74       168

    accuracy                           0.67       239
   macro avg       0.65      0.67      0.65       239
weighted avg       0.72      0.67      0.69       239

rbf:                precision    recall  f1-score   support

           0       0.51      0.59      0.55        71
           1       0.81      0.76      0.78       168

    accuracy                           0.71       239
   macro avg       0.66      0.67      0.66       239
weighted avg       0.72      0.71      0.71       239



## 4) Collaborative Statement (5 points)
#### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

Even if you did not use any outside resources or collaborate with anyone, please state that explicitly in the space below.

I worked alone in this assignment.